In [1]:
import cv2
import torch
import albumentations as al
from albumentations.pytorch import ToTensorV2
from time import time
import numpy as np

In [7]:
model = m.Model4(8, 3)
model.load_state_dict(torch.load(r'logging/XP20_model'))
model.eval()

Model4(
  (layer1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
  )
  (layer4): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.2

In [8]:
data_transform_t = al.Compose([
    al.Resize(120, 160, always_apply=True),
    al.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(transpose_mask=True)
])

lst = ['right FIST',
       'left FIST',
       'right PALM',
       'left PALM',
       'right 2FINGER',
       'left 2FINGER',
       'right L',
       'left L']

In [9]:
sensitivity = 5
confidence = .80
vid = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_PLAIN
prev = 0
frames = np.empty((sensitivity, 1))
frames.fill(np.nan)
fbs = 12
while True:
    ret, frame = vid.read()
    prev = time() - prev
    if time() - prev > 1/fbs:
        frame_t = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_t = data_transform_t(image=frame_t)['image']
        with torch.no_grad():
            out = model(frame_t.unsqueeze(0)).softmax(-1)
            predict = out.argmax(axis=-1).item() if out.max().item() > confidence else np.nan
            frames = np.append(frames[1:], predict)
            print(frames)
    text = f'{lst[int(frames[-1])]}, confidence: {out.max().item():4f}'  if np.all(frames[-1] == frames) else 'undetected'
    cv2.putText(frame,
                text,
                (50, 50),
                font, 1,
                (0, 0, 255),
                2,
                cv2.LINE_4)
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

vid.release()
cv2.destroyAllWindows()

error: OpenCV(4.5.5) D:\bld\libopencv_1646358785966\work\modules\highgui\src\window.cpp:1000: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'


In [ ]:
vid.release()
cv2.destroyAllWindows()

In [15]:
from dataset import HandGestures
from torch.utils.data import DataLoader

In [17]:
data = HandGestures(r'C:\Users\Aboud\Datasets\NUS_hand_posture_data_2\Data')
dl = DataLoader(data, batch_size=256)

In [3]:
predicated = []
target = []

for i, l in dl:
    outputs = model(i)
    outputs = outputs.softmax(-1).argmax(-1)
    predicated += outputs.flatten().tolist()
    target += l.flatten().tolist()

correct = 0
length = len(target)
for i, j in zip(target, predicated):
    if i == j:
        correct += 1
print(correct/length)

NameError: name 'model' is not defined

In [60]:
from PIL import Image
"""trans_r = al.Compose([

    ,
              al.FancyPCA(),
              ,
    al.OneOf([,
              ,
              al.ISONoise(color_shift=(.01, .10), intensity=(.1, .6), p=.25)], p=.5),
    al.GridDistortion(num_steps=15, distort_limit=.5,
                      interpolation=cv2.INTER_LINEAR, border_mode=cv2.BORDER_REPLICATE),
    )
])"""
d = [x[0] for x in data[140:144]]
d = np.array(d)
t = torch.from_numpy(d)
t: torch.Tensor = t.permute(0 ,3, 1, 2).to(torch.float32)
torch.max(t, 2)[0].max(2)[0]


tensor([[206., 207., 209.],
        [255., 255., 255.],
        [255., 255., 249.],
        [201., 195., 198.]])

In [64]:
len(dl.dataset)

3544

In [3]:
ml = vlc.MediaListPlayer()

In [14]:
d

NameError: name 'd' is not defined

In [ ]:
ml.play()

In [ ]:
ml.pause()

In [ ]:
ml.is_playing()

In [2]:
from models import *
v4 = Var4(8)
sum(p.numel() for p in v4.parameters() if p.requires_grad)

1007400

In [19]:
v4 = Var7(8)
v4.eval()
sum(p.numel() for p in v4.parameters() if p.requires_grad)

1731816

In [36]:
input_names = ["actual_input"]
output_names = ["output"]


torch.onnx.export(v4, torch.randn(1, 3, 120, 160), "var7.onnx", verbose=True, input_names=input_names, output_names=output_names, export_params=False, opset_version=11)

graph(%actual_input : Float(1, 3, 120, 160, strides=[57600, 19200, 160, 1], requires_grad=0, device=cpu),
      %fc.0.weight : Float(8, 512, strides=[512, 1], requires_grad=1, device=cpu),
      %fc.0.bias : Float(8, strides=[1], requires_grad=1, device=cpu),
      %525 : Float(32, 3, 3, 3, strides=[27, 9, 3, 1], requires_grad=0, device=cpu),
      %526 : Float(32, strides=[1], requires_grad=0, device=cpu),
      %528 : Float(32, 32, 3, 3, strides=[288, 9, 3, 1], requires_grad=0, device=cpu),
      %529 : Float(32, strides=[1], requires_grad=0, device=cpu),
      %531 : Float(64, 32, 3, 3, strides=[288, 9, 3, 1], requires_grad=0, device=cpu),
      %532 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %534 : Float(128, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cpu),
      %535 : Float(128, strides=[1], requires_grad=0, device=cpu),
      %537 : Float(128, 128, 3, 3, strides=[1152, 9, 3, 1], requires_grad=0, device=cpu),
      %538 : Float(128, strides=[1], r

In [4]:
from models import *
Var1(8)

Var1(
  (layer1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer2): Sequential(
    (0): Conv2d(32, 48, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer4): Sequential(
    (0): Conv2d(48, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer5): Sequential(
    (0): Conv2d(64, 96, kernel_size